In [1105]:
import numpy as np 
import pandas as pd 
import scipy.io

import matplotlib.pyplot
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [1106]:
# Preparing the dataset

data = scipy.io.loadmat('D:/Sahil/Projects/swd/NNFL/Assignment 3/data.mat')
data = np.array(data['ecg_in_window'])
data = data/np.max(data)

labels = scipy.io.loadmat('D:/Sahil/Projects/swd/NNFL/Assignment 3/labels.mat')
labels = np.array(labels['label'])

final_data = np.zeros([1000, 1001])

for i in range(1000):
    final_data[:,i] = data[:,i]

final_data[:,-1] = labels[:,-1]

np.random.shuffle(final_data)

final_data = np.array([final_data])
final_data = np.reshape(final_data, (1000,1001))

In [1107]:
train_data, test_data = final_data[:int(0.7*len(final_data))], final_data[int(0.7*len(final_data)):]

In [1108]:
# Data Loader
class SignalsDataset(Dataset):

    def __init__(self, data, transform=None):
        self.dataframe = data
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        
        sample = {}
        
        sample['data'] = np.reshape(self.dataframe[idx,:1000], (1, 1000))
        sample['target'] = np.array(self.dataframe[idx,1000])
        
        sample['data'] = torch.from_numpy(sample['data'])
        sample['target'] = torch.from_numpy(sample['target'])
        
        if self.transform:
            sample = self.transform(np.array([sample]))

        return sample

In [1109]:
batch_size = 4

In [1110]:
train_data_obj = SignalsDataset(data=train_data)
test_data_obj = SignalsDataset(data=test_data)

train_loader = DataLoader(train_data_obj, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_data_obj, batch_size=batch_size, shuffle=False, num_workers=0)

In [1111]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.cnn_layers = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=6, stride=1),
            nn.AvgPool1d(kernel_size=2, stride=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1),
            nn.AvgPool1d(kernel_size=2, stride=2),
            nn.BatchNorm1d(128),
            nn.ReLU())
        
        self.linear = nn.Sequential(
            nn.Linear(31616, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid())
        
    def forward(self, x):
        
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1) 
        x = self.linear(x)
        x = torch.squeeze(x)
        return x

In [1112]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()
model = model.to(device)
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv1d(1, 64, kernel_size=(6,), stride=(1,))
    (1): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
    (5): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
  )
  (linear): Sequential(
    (0): Linear(in_features=31616, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [1113]:
def train(epoch):
    model.train()
    
    for i, sample in enumerate(train_loader):
        data = sample['data'].to(device)
        target = sample['target'].to(device) 
        data = data.float()
        target = target.float()
        
        optimizer.zero_grad()
        output = model(data)
        
        loss = criterion(output, target)
        loss.backward()
        
        optimizer.step()
        
    print("Training epoch {}".format(epoch+1))

In [1114]:
# Testing function
def test():
        
        model.eval()
        
        test_loss = 0
        correct = 0
        
        for i, sample in enumerate(test_loader):

            data = sample['data'].to(device)
            target = sample['target'].to(device) 
            data = data.float()
            target = target.float()
            
            output = model(data)
            
            for j in range(len(target)):
                out = 0 if output[j].item() < 0.5 else 1
#                 print(out, target[j].item())
                if target[j].item() == out:
                    correct += 1
        
        print("Validation accuracy: ", correct/len(test_data))

In [1115]:
for epoch in range(20):
    train(epoch)
    with torch.no_grad():
        test()

Training epoch 1
Validation accuracy:  0.6633333333333333
Training epoch 2
Validation accuracy:  0.7466666666666667
Training epoch 3
Validation accuracy:  0.7933333333333333
Training epoch 4
Validation accuracy:  0.7866666666666666
Training epoch 5
Validation accuracy:  0.7933333333333333
Training epoch 6
Validation accuracy:  0.8066666666666666
Training epoch 7
Validation accuracy:  0.8333333333333334
Training epoch 8
Validation accuracy:  0.8066666666666666
Training epoch 9
Validation accuracy:  0.8133333333333334
Training epoch 10
Validation accuracy:  0.8133333333333334
Training epoch 11
Validation accuracy:  0.8033333333333333
Training epoch 12
Validation accuracy:  0.82
Training epoch 13
Validation accuracy:  0.8
Training epoch 14
Validation accuracy:  0.83
Training epoch 15
Validation accuracy:  0.8166666666666667
Training epoch 16
Validation accuracy:  0.7833333333333333
Training epoch 17
Validation accuracy:  0.8033333333333333
Training epoch 18
Validation accuracy:  0.8033333